In [2]:
import tkinter as tk
from tkinter import ttk
import tkinter.messagebox
import pinyin
import pandas as pd
import string
import sys
import time
import logging
import difflib
import sys
from ttkbootstrap import Style
import csv
import getopt
import argparse
import requests
import parsel
from lxml import etree
from tkinter import filedialog

def fuzzy_matching(texts, value):
    texts = texts.split(",")
    texts_score = {}
    logging.info("展开模糊搜索成功！\n")
    for i in texts:
        score = difflib.SequenceMatcher(None, i, value).quick_ratio()
        texts_score[i] = score
    texts_score = sorted(texts_score.items(), key=lambda x: x[1], reverse=False)
    logging.info("获取搜索匹配数据成功，且对数据进行排序！\n")
    po=1
    tmp12=0
    if flagofsearch==1 :
        b=[x[1] for x in texts_score]
        for i in range(100):
            if(b[-i]>0):
                print(i,end=':')
                print(texts_score[-i])
                tmp12=1
        logging.info("成功输出获取数据供给用户选择！(开启了模糊匹配选项)\n")
        if(tmp12==1):
            po=int(input("选择你所想输入的城市名称（1-N）:"))
            match_value = texts_score[-po][0]
            return match_value
        else:
            return -1    
    else :
        return texts_score[-1][0]
def get_pinyin_first_alpha(name):
    return "".join([i[0] for i in pinyin.get(name, " ").split(" ")])


class House:
    def __init__(self,master):
        self.columns={
        'ID':25,
        'city':40,
        'title':350,
        'address':200,
        'price':40,
        'price_pre_squ':100,
        'houseInfo':450
        }
        self.tree=ttk.Treeview(master,height=18,show='headings')
        self.tree['columns'] = list(self.columns)
        for column in self.columns:  # 批量设置列属性
            self.tree.heading(column, text=column)  # #设置列标题
            self.tree.column(column, width=self.columns[column], anchor='center')  
        
        self.progressbar=ttk.Progressbar(master,length=800)
        self.progressbar.grid(row=3,column=0,columnspan=3,pady=10)
        self.progressbar['maximum']=100
        # self.canvas=tk.Canvas(master,width=600,height=15,bg='black')
        # self.canvas.grid(row=3,column=0,columnspan=3,pady=10)


        self.tree.grid(row=1,column=0,columnspan=3) 
       

        self.b1 = tk.Label(master,text='请输入省会城市的名称:',font=('微软雅黑',12))
        self.b1.grid(row=0,column=0,padx=10)

        self.e =tk.Entry(master,show=None,width=100)
        self.e.grid(row=0,column=1,padx=10) 

        # self.bt1=tk.Button(master,text='搜索',width=8,height=1,command=lambda:[self.progress(),self.search()])
        self.bt1=tk.Button(master,text='搜索',width=8,height=1,command=lambda:[self.showprogress(),self.search()])

        self.bt1.grid(row=0,column=2,padx=10)  
        self.btnDelete1=tkinter.Button(master,text='删除所选数据',font=('微软雅黑',12),command=self.deleteclick,width=10,height=1)
        self.btnDelete1.grid(row=2,column=0)  #
        self.btnDelete2=tkinter.Button(master,text='删除所有数据',font=('微软雅黑',12),command=self.deletealldata,width=10,height=1)
        self.btnDelete2.grid(row=2,column=1)
        self.button4 = tk.Button(master,text='保存所有数据',font=('微软雅黑',12),command=self.save)
        self.button4.grid(row=2,column=2)

    def deleteclick(self):
        if not self.tree.selection():
            tkinter.messagebox.showerror('抱歉','你还没有选择，无法删除数据')
            return
        for item in self.tree.selection():
            self.tree.delete(item)

    def showprogress(self):
        for i in range(100):
            self.progressbar['value']=i+1
            window.update()
            time.sleep(0.005)
    
    # def progress(self):
    #     # 填充进度条
    #     fill_line = self.canvas.create_rectangle(1.5, 1.5, 0, 23, width=0, fill="green")
    #     x = 500  # 未知变量，可更改
    #     n = 300 / x  # 465是矩形填充满的次数
    #     for i in range(x):
    #         n = n + 465 / x
    #         self.canvas.coords(fill_line, (0, 0, n, 60))
    #         window.update()
    #         time.sleep(0.0000000001)  # 控制进度条流动的速度
    
    #     # 清空进度条
    #     fill_line = self.canvas.create_rectangle(1.5, 1.5, 0, 23, width=0, fill="white")
    #     x = 500  # 未知变量，可更改
    #     n = 300 / x  # 465是矩形填充满的次数
    
    #     for t in range(x):
    #         n = n + 465 / x
    #         # 以矩形的长度作为变量值更新
    #         self.canvas.coords(fill_line, (0, 0, n, 60))
    #         window.update()
    #         time.sleep(0)  # 时间为0，即飞速清空进度条
  


    def deletealldata(self):
        global cnt
        def delButton(tree):
            x=tree.get_children()
            for item in x:
                # print(item)
                tree.delete(item)
            logging.info('删除所有数据成功')
        delButton(self.tree)
        cnt=0
    

    def search(self):
        global city
        global cnt
        self.progressbar['value']=0
        city =self.e.get()
        if city:
            titles = []
            addresses = []
            introduces = []
            totalprices = []
            unitprices = []
            ress = []
            res=fuzzy_matching(data1,city)
            if res!=-1:
                logging.info("模糊匹配完成！")
                resultcity=get_pinyin_first_alpha(res)
                logging.info("汉字拼音首字母获取成功！")
                url='https://'+resultcity+'.lianjia.com/ershoufang/'
                headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.42'}
                reponse = requests.get(url=url,headers=headers)
                html_data = reponse.text
                #print(html_data)
                selector = parsel.Selector(html_data)
                lis = selector.css('.clear.LOGCLICKDATA')
                #print(lis)
                for li in lis:
                    title = li.css('.title a::text').get()
                    cnt+=1
                    id=str(cnt)
                    address = '-'.join(li.css('.positionInfo a::text').getall())
                    introduce = li.css('.houseInfo::text').get()
                    
                    totalprice = li.css('.priceInfo .totalPrice span::text').get()+'万'
                    unitprict = li.css('.unitPrice span::text').get()
                    Data=[id,res,title,address,totalprice,unitprict,introduce]
                    titles.append(title)
                    addresses.append(address)
                    introduces.append(introduce)
                    totalprices.append(totalprice)
                    unitprices.append(unitprict)
                    ress.append(res)
                    datas=pd.DataFrame([ress,titles,addresses,totalprices,unitprices,introduces]).T
                    self.tree.insert('', cnt,text='', values=Data)
                    window.update()
            
                datas.columns=['city','title','address','price','price_pre_squ','houseInfo']
                logging.info('爬取信息成功')
                return datas 
            else:
                tkinter.messagebox.showerror('错误',"没有找到相应的城市！可重新启动程序在初始化菜单选择4进行补充！")
        else :
            tkinter.messagebox.showinfo(title = '错误信息',message = '请输入内容')
    def save(self):
        self.df = self.search()
        try:
            savefile = filedialog.asksaveasfilename(filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*") ))
            self.df.to_excel(savefile + ".xlsx", index=False, sheet_name="Results")
            logging.info('保存数据成功')
        except Exception as e:
            tkinter.messagebox.showerror(title = '错误信息',message = str(e))






flagofsearch=0  #两个flag
file_name="./城市数据.txt"
filename123="./logging.txt"
logging.basicConfig(filename=filename123,level=logging.INFO,format="%(asctime)s-%(levelname)s:%(message)s")
print("****************************初始化菜单****************************")
print("1:重新输入城市数据文件路径")
print("2:取消模糊匹配支持（输入城市名称后仅输出认为最大匹配度的城市）")
print("3:启用模糊匹配")
print("4：追加补充城市名称")
print("5：退出菜单")
print("*****************************************************************")
tmpp=tmpp=int(input("输入1-5选择所要执行的"))
while tmpp!=4 :
    if(tmpp==1):
        file_name=input("请输入新的文件路径")
        logging.info("重新选取了数据文件路径")
    elif(tmpp==2):
        flagofsearch=0
        logging.info("取消了模糊匹配")
        print("模糊匹配已取消")
    elif(tmpp==3):
        flagofsearch=0
        logging.info("开启了模糊匹配")
        print("模糊匹配已开启")
    elif(tmpp==4)  :
        cityname=input("请输入城市名称:")
        f1= open(file_name,'a',encoding='utf-8')
        f1.write(','+cityname)
        f1.close()
    elif(tmpp==5):
        logging.info("初始化结束！")
        print("初始化结束！")
        break
    else :
        print("输入错误！请重新输入！")
    tmpp=int(input("输入1-5选择所要执行的"))

f = open(file_name,encoding='utf-8')
data1=f.read()

t = 20
print("*******************************加载城市数据*******************************")
start = time.perf_counter()
for i in range(t + 1):
    finsh = "▓" * i
    need_do = "-" * (t - i)
    progress = (i / t) * 100
    dur = time.perf_counter() - start
    print("\r{:^3.0f}%[{}->{}]{:.2f}s".format(progress, finsh, need_do, dur), end="")
    time.sleep(0.05)
logging.info("进度条加载完毕！")

window=tk.Tk()
# style = Style(theme='pulse')
# window=style.master
city=''
cnt=0

window.title('省会二手房信息搜索')
window.geometry('1265x530')

window.tk.call("source", "azure.tcl")
window.tk.call("set_theme", "light")

window.maxsize(1300,540)
window.minsize(1265,500)
House(window)
window.mainloop()




****************************初始化菜单****************************
1:重新输入城市数据文件路径
2:取消模糊匹配支持（输入城市名称后仅输出认为最大匹配度的城市）
3:启用模糊匹配
4：追加补充城市名称
5：退出菜单
*****************************************************************
初始化结束！
*******************************加载城市数据*******************************
100%[▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓->]1.24s

In [3]:
import cv2
import numpy as np
import socket
import struct
import pickle

# 连接到服务器
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('0.0.0.0', 12345))

# 接收数据缓冲区
data = b''
payload_size = struct.calcsize(">L")

while True:
    # 接收图像数据
    while len(data) < payload_size:
        data += client_socket.recv(4096)

    packed_msg_size = data[:payload_size]
    data = data[payload_size:]
    msg_size = struct.unpack(">L", packed_msg_size)[0]

    while len(data) < msg_size:
        data += client_socket.recv(4096)

    frame_data = data[:msg_size]
    data = data[msg_size:]

    # 将接收到的图像数据解码为OpenCV图像
    frame = pickle.loads(frame_data)
    img = cv2.imdecode(frame, cv2.IMREAD_COLOR)

    # 显示图像
    cv2.imshow('Received Image', img)
    if cv2.waitKey(1) == ord('q'):
        break

# 关闭套接字连接和窗口
client_socket.close()
cv2.destroyAllWindows()


gaierror: [Errno 11001] getaddrinfo failed